In [7]:
import pandas as pd
import json
from tqdm import tqdm

In [2]:
a = []

f = open('20220511T174759.log')
line = f.readline()
while line:
    line = f.readline()
    a.append(line)
    
f.close()    

In [6]:
a[0]

'11052022:160119;cherepovets;ak1456_1881;#;bus;59.1339225769043;37.97321701049805;0.00;0;0;;е254хк_35 [БВ (Н)];\n'

In [13]:
df = pd.DataFrame([sub.split(":") for sub in a])
df.columns = ['date', 'data']
df = df[df.date != '']

In [45]:
df.shape

(67600, 2)

In [40]:
%%time
df_append = []

for i in range(len(df.data)):
    try:
        df_append.append(pd.DataFrame(df.data[i].split(";")).T)
    except AttributeError:
        pass
    
df_append = pd.concat(df_append)    

df_append.shape

CPU times: user 22.6 s, sys: 168 ms, total: 22.8 s
Wall time: 22.7 s


(67600, 13)

In [60]:
df_test_date = df.copy(deep=True)
df_test_data = df_append.copy(deep=True)

In [61]:
df_test_data = df_test_data.reset_index(drop=True)

In [66]:
df_main = pd.concat([df_test_date, df_test_data],axis=1)

In [70]:
df_main.head()

,date,data,0,1,2,3,4,5,6,7,8,9,10,11,12
0,11052022,160119;cherepovets;ak1456_1881;#;bus;59.133922...,160119,cherepovets,ak1456_1881,#,bus,59.1339225769043,37.97321701049805,0.00,0,0,,е254хк_35 [БВ (Н)],\n
1,11052022,160117;cherepovets;ak1456_2424;#7;bus;59.12198...,160117,cherepovets,ak1456_2424,#7,bus,59.12198257446289,37.97989273071289,9.44,8,1,,е401хк_35 [БВ (Н)],7 [ул. олимпийская - > - ул. наседкина]\n
2,11052022,160117;cherepovets;ak1456_1946;#27;bus;59.0967...,160117,cherepovets,ak1456_1946,#27,bus,59.096702575683594,37.9358024597168,28.49,280,1,,е387хк_35 [БВ (Н)],27 [ул. монт - клер - > - доменная]\n
3,11052022,160118;cherepovets;ak1456_1892;#8;bus;59.12646...,160118,cherepovets,ak1456_1892,#8,bus,59.126468658447266,37.91294479370117,17.76,103,1,,е329хк_35 [БВ (Н)],8 [доменная - > - ул. красная]\n
4,11052022,160115;cherepovets;ak1456_2408;#18;bus;59.1187...,160115,cherepovets,ak1456_2408,#18,bus,59.1187744140625,37.90309143066406,0.00,13,1,,е407хк_35 [БВ (Н)],18 [ул. рыбинская (2) -> пр. победы 2]\n


In [75]:
df_draw = df_main[['date',5,6]]

In [76]:
new_name_columns = ['date','latitude','longitude']

In [77]:
df_draw.columns.str.replace(new_name_columns)

TypeError: replace() missing 1 required positional argument: 'repl'

In [ ]:
headers = ['latitude', 'longitude']

df_coordinates = pd.DataFrame(coordinates, columns = headers)
df_coordinates